In [19]:
from googletrans import Translator
from os import chdir, getcwd, listdir, makedirs
from os.path import join,splitext
from docx import Document
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
import fitz
import pytesseract
from pytesseract import image_to_string
from PIL import Image
from PyPDF2 import PdfReader
import io
from pdf2image import convert_from_path
import re

In [20]:
current_dir = getcwd()

In [21]:
target_folder = join(current_dir, r"C:\Users\acer\Desktop\test")
print("Target Folder Path:", target_folder)

Target Folder Path: C:\Users\acer\Desktop\test


In [91]:
# List files and subdirectories within a folder
file_list = listdir(target_folder)
print("Files in Target Folder:", file_list)

Files in Target Folder: ['Data Analytics assignment 1questions - Copy (2).docx', 'maths syllabus.pdf', '_Web Based Learning Management System.docx']


In [45]:
stopwords = list(STOP_WORDS)
nlp = spacy.load('en_core_web_sm')
content = ""

In [34]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [35]:
def get_Language(argument):
    switcher = {
        0: "en",
        6: "hi",
        5: "zh-CN",
        4: "ko",
        3: "ja",
        1: "de"
    }
    return switcher.get(argument, "en")

In [36]:
def getSelection():
    while True:
        try:
            userInput = int(input())
            if userInput < 0 or userInput > 6:
                print("Not an integer! Try again.")
                continue
        except ValueError:
            print("Not an integer! Try again.")
            continue
        else:
            return userInput
            break

In [37]:
translator = Translator()

In [93]:
print("0. ENGLISH")
print("6. HINDI")
print("5. Chinese")
print("4. KOREAN")
print("3. JAPANESE")
print("1. GERMAN")
tran = getSelection()
lang = get_Language(tran)
print("Selected Language:", lang)

0. ENGLISH
6. HINDI
5. Chinese
4. KOREAN
3. JAPANESE
1. GERMAN
4
Selected Language: ko


In [94]:
for i in file_list:
    path = r"C:\Users\acer\Desktop\test\{}".format(i)
    def get_file_extension(path):
        return splitext(path)[1][1:]  # Remove the leading dot
    # Example usage:
    extension = get_file_extension(path)
    #print("File extension:", extension)
    if(extension == 'docx'):
        # Load the Word document
        doc = Document(path)
        # Read the content of the document
        content:str = ''
        page_content = ""
        content = "\n".join([para.text for para in doc.paragraphs])
        page_content += content
        preprocessed_content = re.sub(r'\s+', ' ', page_content).strip
        translated_text = translator.translate(page_content, dest=lang)
        print(translated_text.text)
    else:
        def check_pdf_content(pdf_path):
            has_text = False
            has_images = False
            with open(pdf_path, 'rb') as pdf_file:
                pdf_reader = PdfReader(pdf_file)
                # Check if the PDF contains text
                for page_num in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[page_num]
                    page_text = page.extract_text()
                    if page_text.strip():  # Check if the extracted text is not empty
                        has_text = True
                        break
                  # Check if the PDF contains images
            if '/XObject' in page['/Resources']:
                xObject = page['/Resources']['/XObject'].get_object()
                if xObject:
                    for obj in xObject:
                         if xObject[obj]['/Subtype'] == '/Image':
                            has_images = True
                            break
            return has_text, has_images
        if __name__ == "__main__":
            text_present, images_present = check_pdf_content(path)

            if text_present:
                # If the page has text, extract it directly
                with open(path, 'rb') as pdf_file:
                    # Create a PDF reader object
                    pdf = PdfReader(pdf_file)
                    # Get the number of pages in the PDF
                    num_pages = len(pdf.pages)
                    # Extract text from all pages and store it in the 'text' variable
                    
                    for page_num in range(num_pages):
                        page = pdf.pages[page_num]
                        content:str = ''
                        page_content = ""
                        content += page.extract_text()
                        page_content += content
                        preprocessed_content = re.sub(r'\s+', ' ', page_content).strip
                        translated_text = translator.translate(page_content, dest=lang)
                        print(translated_text.text)
                #print("The PDF contains text.")

            elif images_present:
                # Open the PDF file and read its content using PyPDF2
                with open(path, 'rb') as pdf_file:
                    pdf_reader = PdfReader(pdf_file)
                    # Iterate through all pages
                    for page_num in range(len(pdf_reader.pages)):
                        # Convert the page to an image using pdf2image
                        images = convert_from_path(path, first_page=page_num + 1, last_page=page_num + 1, poppler_path = r"C:\Users\acer\Downloads\Release-23.07.0-0\poppler-23.07.0\Library\bin")
                        # Initialize the page_content variable to store text extracted from this page
                        content:str = ''
                        page_content = ""
                        # Save each image to the output folder
                        for idx, image in enumerate(images):
                            extracted_text = pytesseract.image_to_string(Image.open(join(f"page{page_num + 1}_image{idx + 1}.jpg")))
                            page_content += extracted_text
                            preprocessed_content = re.sub(r'\s+', ' ', page_content).strip
                            translated_text = translator.translate(page_content, dest=lang)
                            print(translated_text.text)
                            #print("Detected Text:", preprocessed_content)

데이터 분석 할당 질문
질문 1.상세한 예를 제공하는 샘플링 및 샘플링 분포는 무엇입니까?
질문 2.빅 데이터 수명주기의 다양한 단계는 각각을 설명하는 그림으로 설명합니다.
질문 3.통계 수업에서 학생들의 평균 연령은 22 세입니다.이 진술은 설명을 설명합니까?
추론 통계?답변을 자세히 설명하십시오.(ans. 설명 분석)
질문 4.하루에 생산 된 전화의 4% 미만이 결함이있는 경우 조립 라인이 만족스럽게 작동하고 있습니다.에게
하루의 생산의 품질을 확인한 회사는 하루의 프로덕션에서 10 전화를 무작위로 샘플링합니다.
결함을 테스트하십시오.제조업체에 관심있는 인구를 정의하십시오.
a) 10 개의 전화가 샘플링되고 테스트되었습니다
b) 문제의 낮 동안 생산 된 모든 전화
c) 결함이있는 전화의 4%
d) 10 개의 응답 : 결함이 있거나 결함이 없음
(Ans. B 이와 함께 인구를 자세히 설명합니다)
질문 5.남학생의 높이는 일반적으로 평균 170cm와
8cm의 표준 편차.키가있는 남학생의 비율을 찾으십시오.
Ans.평균 = 170 cm (μ);표준 편차 = 8 cm (σ)
(i) 186cm 이상
186 = 170 + 16
170 + 2 (8)
이것은 186cm보다 큰 μ + 2σ 이상의 영역이라는 것을 의미합니다.

2σ 이상의 백분율 = 2 · 35% + 0 · 15% (그래프에서) = 2 · 5%
(ii) 162cm에서 186cm 사이
170-162 = 8
162 = μ - σ
162에서 186 사이의 면적은 μ - σ → μ + 2σ의 면적입니다.

μ - σ → μ + 2σ의 백분율 = 68% + 13 · 5% (그래프에서) = 81 · 5%
(iii) 154cm에서 178cm 사이.
170-154 = 16
= 2 (8)
154 = μ -2σ
178-170 = 8
178 = μ + σ
154에서 178 사이의 면적은 μ - 2σ → μ + σ의 면적입니다.

μ -2σ → μ + σ = 13 · 5% + 68% = 81 · 5%에서 비율
질문 6. 대상 

### 